<a href="https://colab.research.google.com/github/drustantmetar/DataFrame1/blob/main/Dataframe2SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# load the libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType,StringType,StructType,StructField,DoubleType

In [2]:
# create spark session
spark= SparkSession\
.builder\
.master("local")\
.appName("Data Frame 2")\
.getOrCreate()

In [4]:
# get the data from data source to crete Data Frame for DSL : Domain Specific langauge
moviesdf=spark.read.format("csv").option("inferschema","true").option("header","true").load("/content/movies.csv")
ratingsdf=spark.read.format("csv").option("inferschema","true").option("header","true").load("/content/ratings.csv")

In [6]:
print("movie data frame schema")
moviesdf.printSchema()
print("ratigns data frame schema")
ratingsdf.printSchema()

movie data frame schema
root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

ratigns data frame schema
root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [9]:
moviesdf.schema

StructType([StructField('movieId', IntegerType(), True), StructField('title', StringType(), True), StructField('genres', StringType(), True)])

In [12]:
ratingsdf.schema

StructType([StructField('userId', IntegerType(), True), StructField('movieId', IntegerType(), True), StructField('rating', DoubleType(), True), StructField('timestamp', IntegerType(), True)])

In [21]:
# To create our own schema for movie data frame
movieschema =  StructType(
 [
	StructField('movieId', IntegerType(), True),
	StructField('title', StringType(), True),
	StructField('genres', StringType(), True)
 ]
 )

In [30]:
# To use this created schema into our created data frame
moviesdf=spark.read.format("csv").schema(movieschema).option("inferschema","true").load("/content/movies.csv")

In [27]:
moviesdf.schema

StructType([StructField('movieId', IntegerType(), True), StructField('title', StringType(), True), StructField('genres', StringType(), True)])

In [32]:
moviesdf.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [33]:
# To create own schema for rating data frame
ratingsdf.schema

StructType([StructField('userId', IntegerType(), True), StructField('movieId', IntegerType(), True), StructField('rating', DoubleType(), True), StructField('timestamp', IntegerType(), True)])

In [37]:
ratingschema= StructType(
   [
   StructField('userId', IntegerType(), True),
   StructField('movieId', IntegerType(), True),
   StructField('rating', DoubleType(), True),
   StructField('timestamp', IntegerType(), True)
   ]
   )


In [58]:
# add the rating schema into the ratings schema
ratingsdf=spark.read.format("csv").schema(ratingschema).load("/content/ratings.csv")

In [59]:
ratingsdf.schema

StructType([StructField('userId', IntegerType(), True), StructField('movieId', IntegerType(), True), StructField('rating', DoubleType(), True), StructField('timestamp', IntegerType(), True)])

In [60]:
ratingsdf.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [61]:
movieFilterdf=moviesdf.filter("genres == 'Comedy'")

In [69]:
joindf=movieFilterdf.join(ratingsdf, on= "movieId")


In [82]:
joindf.groupBy("title").agg(count("*").alias("Count of Rating")).orderBy(col("Count of Rating").desc()).show(10)

+--------------------+---------------+
|               title|Count of Rating|
+--------------------+---------------+
|Ace Ventura: Pet ...|          28483|
|       Clerks (1994)|          17713|
|Ferris Bueller's ...|          17341|
|Birdcage, The (1996)|          16477|
|Ace Ventura: When...|          15613|
|Monty Python's Li...|          15397|
|    Airplane! (1980)|          14129|
|Happy Gilmore (1996)|          13382|
|    Liar Liar (1997)|          12633|
|This Is Spinal Ta...|          12049|
+--------------------+---------------+
only showing top 10 rows



In [85]:
moviesdf.printSchema()
ratingsdf.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



**Below Code which is written in single line**

In [94]:
moviesdf.filter("genres == 'Comedy'").join(ratingsdf, on="movieId").groupBy("title").agg(count("*").alias("Count of Rating")).orderBy(col("Count of Rating").desc()).show(10)

+--------------------+---------------+
|               title|Count of Rating|
+--------------------+---------------+
|Ace Ventura: Pet ...|          28483|
|       Clerks (1994)|          17713|
|Ferris Bueller's ...|          17341|
|Birdcage, The (1996)|          16477|
|Ace Ventura: When...|          15613|
|Monty Python's Li...|          15397|
|    Airplane! (1980)|          14129|
|Happy Gilmore (1996)|          13382|
|    Liar Liar (1997)|          12633|
|This Is Spinal Ta...|          12049|
+--------------------+---------------+
only showing top 10 rows

